In [1]:
#!pip3 install folium --upgrade
# upgrade folium to version 0.12.1
import pandas as pd

In [2]:
df = pd.read_pickle("indeed_job_ca_clean2.pkl")

In [3]:
# This template below allows us to have a legend on our first folium map. 
# Reference https://nbviewer.jupyter.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd
from branca.element import Template, MacroElement
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script src="js/leaflet-providers.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:blue;opacity:0.7;'></span>Not Specified</li>
    <li><span style='background:yellow;opacity:0.7;'></span>Bachelor\'s Degree</li>
    <li><span style='background:red;opacity:0.7;'></span>Master\'s Degree</li>
    <li><span style='background:green;opacity:0.7;'></span>Doctoral Degree</li>


    
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

In [4]:
import folium
import copy
from folium.plugins import MarkerCluster

# We take a copy of the original data frame
df_geo = copy.deepcopy(df)

# We create a list for each row in the dataframe.
geo_lst = df_geo.values.tolist()

# We create an empty list.
long_lat_lst = []

# We loop through the list and only append the long and lat values.
for i in geo_lst:
    long_lat_lst.append((i[-1],i[-2]))
    
# We draw a folium map.
m = folium.Map(location=[37.1915, -119.9889],
                        tiles='https://tiles.stadiamaps.com/tiles/alidade_smooth/{z}/{x}/{y}{r}.png', attr='Stadia.AlidadeSmooth',
                        zoom_start = 6)



for j in range(len(long_lat_lst)):
        
        # We grab the infos that we want to add to the popup.
        title = df_geo.iloc[j, df_geo.columns.get_loc("Title")]
        company = df_geo.iloc[j, df_geo.columns.get_loc("Company")]
        location = df_geo.iloc[j, df_geo.columns.get_loc("Location")]
        min_edu = df_geo.iloc[j, df_geo.columns.get_loc("Minimum Education")]
        popup_text = "Job Title: {title} <br>""Company: {company} <br>""Location: {location} <br>""Minimum Education: {min_edu} <br>".format(title=title,company=company, location=location, min_edu = min_edu)

        
        # We check adjust the color of the of the data point based on the degree.
        if df_geo.iloc[j, df_geo.columns.get_loc("Minimum Education")] == 'Not Specified':
            fill_col = 'blue'
        # Else the color will be red.
        elif df_geo.iloc[j, df_geo.columns.get_loc("Minimum Education")] == 'Bachelor\'s Degree':
            fill_col = 'yellow'
        elif df_geo.iloc[j, df_geo.columns.get_loc("Minimum Education")] == 'Master\'s Degree':
            fill_col = 'red'
        elif df_geo.iloc[j, df_geo.columns.get_loc("Minimum Education")] == 'Doctoral Degree':
            fill_col = 'green'
        
        # We create the circles in the map. 
        folium.Circle(
        radius=100,
        location = [long_lat_lst[j][1],long_lat_lst[j][0]],
        color = fill_col,
        popup = folium.Popup(popup_text, max_width=300,min_width=300),
        fill = True).add_to(m)
        
        
#         folium.CircleMarker(
#         radius=70,
#         location = [37.9975, -122.1869],
#         color = 'black',
#         fill = False).add_to(m)
        
        
#         folium.CircleMarker(
#         location = [33.9592, -118.4194],
#         radius=95,
#         color='black',
#         fill = False
#         ).add_to(m)

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)
m 

In [5]:
# This template below allows us to have a legend on our second folium map. 
# Reference https://nbviewer.jupyter.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd
template2 = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script src="js/leaflet-providers.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:green;opacity:0.7;'></span>Eligible To Apply</li>
    <li><span style='background:red;opacity:0.7;'></span>Not Eligible To Apply</li>


    
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

In [6]:
# We draw a folium map.
m2 = folium.Map(location=[37.1915, -119.9889],
                        tiles='https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png', attr='Stadia.AlidadeSmoothDark',
                        zoom_start = 6)



for j in range(len(long_lat_lst)):
        
        # We grab the infos that we want to add to the popup.
        title = df_geo.iloc[j, df_geo.columns.get_loc("Title")]
        company = df_geo.iloc[j, df_geo.columns.get_loc("Company")]
        location = df_geo.iloc[j, df_geo.columns.get_loc("Location")]
        min_edu = df_geo.iloc[j, df_geo.columns.get_loc("Minimum Education")]
        popup_text = "Job Title: {title} <br>""Company: {company} <br>""Location: {location} <br>""Minimum Education: {min_edu} <br>".format(title=title,company=company, location=location, min_edu = min_edu)

        
        # We check adjust the color of the of the data point based on the degree.
        if df_geo.iloc[j, df_geo.columns.get_loc("Minimum Education")] == 'Not Specified':
            fill_col = 'green'
        # Else the color will be red.
        elif df_geo.iloc[j, df_geo.columns.get_loc("Minimum Education")] == 'Bachelor\'s Degree':
            fill_col = 'green'
        elif df_geo.iloc[j, df_geo.columns.get_loc("Minimum Education")] == 'Master\'s Degree':
            fill_col = 'green'
        elif df_geo.iloc[j, df_geo.columns.get_loc("Minimum Education")] == 'Doctoral Degree':
            fill_col = 'red'
        
        # We create the circles in the map. 
        folium.Circle(
        radius=100,
        location = [long_lat_lst[j][1],long_lat_lst[j][0]],
        color = fill_col,
        popup = folium.Popup(popup_text, max_width=300,min_width=300),
        fill = True).add_to(m2)
        
        
        
#         folium.CircleMarker(
#         radius=70,
#         location = [37.9975, -122.1869],
#         color = 'black',
#         fill = False).add_to(m)
        
        
#         folium.CircleMarker(
#                     location = [33.9592, -118.4194],
#                     radius=95,
#                     color='black',
#                     fill = False
#                     ).add_to(m)

macro2 = MacroElement()
macro2._template = Template(template2)

m2.get_root().add_child(macro2)
m2